In [108]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import math

In [146]:
mesh    = UnitSquareMesh (10,10)
Qe      = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
Be      = FiniteElement("Bubble",   mesh.ufl_cell(), 4)
Ve      = VectorElement(NodalEnrichedElement(Qe, Be))
element = MixedElement(Ve,Qe)
W       = FunctionSpace(mesh, element)

# Define trial and test functions
w       = Function(W)
(u, p)  = TrialFunctions(W)
(v, q)  = TestFunctions(W)
n       = FacetNormal(mesh)

sub_domains = MeshFunction('size_t', mesh, 1)
ds = Measure('ds', domain=mesh, subdomain_data=sub_domains)
# Construct a spatially-varying permeability matrix (inverse)#
k = "1.0"
k11 = Expression(k, degree = 2)
k12 = Constant(0.0)
k21 = Constant(0.0)
k22 = Expression(k, degree = 2)
K = as_matrix(((k11, k12), (k21, k22)))

# Costruct bilinear and linear parts of equation
a =  (dot(K*u,v) - div(v)*p - div(u)*q)*dx
L = Constant(1)*dot(n,v)*ds     #(1)  + Constant(2)*dot(n,v)*ds(2)# f*dot(n,v)*ds

# Defining subdomains
tol=1E-15

def walls(x, on_boundary):
    return near(x[0],0) or near(x[0], 1) or \
           near(x[1],0) or near(x[1], 1) and on_boundary          # Wall contour




fill_factor  = np.zeros ((11,11))
# array of facet midpoints 
coord      = np.array ([0.025])
coord      = np.append(coord, [np.arange(0.1, 1, 0.1)])
coord      = np.append(coord, [0.975])
print ('Array of facet midpoints:', coord)

yy, xx = np.meshgrid(coord, coord)
x = xx.reshape (121)
y = yy.reshape (121)

# array of facet lenght
lenght     = np.ones (11)/10
lenght[0]  = lenght[0]/2
lenght[-1] = lenght[-1]/2
print ('Array of facet    lenght:', lenght)


def U (dd):
    d = dd/10
    if d==0:
        def b_inj(x):
            return (abs(x[0] - 0.0) < tol) and (abs(x[1] - 0.1)  < tol)         # Injection coordinates  (1,1)
    else:
        def b_inj(x):
            r = math.sqrt(x[0]*x[0] + x[1]*x[1])
            return (r < d)
    if d==0:
        def b_out(x):
            return (abs(x[0] - 0.5) < tol) and (abs(x[1] - 0.7)  < tol)         # Injection coordinates  (1,1)
    else:
        def b_out(x):
        #    return ((x[0]>x_bc) and (x[1]>y_bc))
            r = math.sqrt(x[0]*x[0] + x[1]*x[1])
            return (r > d)# and (x[1] > 0.1))    # Outflow coordinates    (1,3) 
       
    # Create BC
    bc_n     = DirichletBC(W.sub(0), Constant ((0,0)), walls)                       # Neumann on the walls 
    bc_in    = DirichletBC(W.sub(1), Constant (1),     b_inj, method='pointwise')   # Dirichlet (p=1) on injection point (1,1)
    bc_out   = DirichletBC(W.sub(1), Constant (0),     b_out, method='pointwise')   # Dirichlet (p=0) on out point (1,3)

    # Solving equation and plot results
    solve(a == L, w, bcs = [bc_n, bc_in, bc_out])
    (u, p) = w.split()
    
    # Create arrays of coordinates and zero_array of fill_factor
    coor         = mesh.coordinates()         # np.array (121, 2); all coordinates
    
    # flow in control volume from all (4) facets of control volume
    def Q (i,j):
        vertical_facets   = lenght[i] * (u((coord[i] - lenght[i]/2), coord[j])[0] - u((coord[i] + lenght[i]/2), coord[j])[0])
        horizontal_facets = lenght[j] * (u(coord[i], (coord[j] - lenght[j]/2))[1] - u(coord[i], (coord[j] + lenght[j]/2))[1])
        sum = vertical_facets + horizontal_facets
        #(print ('Q in element (',i,',',j,'):',sum ))
        return sum

    # Array of flow values in [i,j] control volume        
    Q_arr = np.zeros ((11,11))                  
    for i in range (11):
        for j in range (11):
            Q_arr[i,j] = Q(i,j)
    Q_arr = np.where (Q_arr<0, 0, Q_arr)
    dt = 1/Q_arr.max()
    print ('Q max: ', Q_arr.max())
    print ('dt:    ',dt)

    # fill factor_i += Q_i *dt / V_i, где V_i — объём ячейки i, у нас у всех ячеек (контрольных объёмов) он одинаковый.
    fill_factor = Q_arr*dt
    fill_factor=fill_factor.reshape(121)

#     plot(u, title = 'Velocity', mesh=mesh)
#     plt.show()
#     plot(p, title = 'Pressure', mesh=mesh)
#     plt.show()
#     fig = go.Figure(data=go.Heatmap(x = x, y = y,z = fill_factor, type = 'heatmap',colorscale = 'Viridis'))
#     fig.update_layout(width = 400, height = 400, autosize = False, title = 'Fill Factor' )
#     fig.show()
    return fill_factor

Array of facet midpoints: [0.025 0.1   0.2   0.3   0.4   0.5   0.6   0.7   0.8   0.9   0.975]
Array of facet    lenght: [0.05 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.05]


In [190]:
U(0)
fill_factor

Solving linear variational problem.Q max:  0.21157248639103562
dt:    
 4.726512492516466


array([1.47761357e-04, 0.00000000e+00, 0.00000000e+00, 5.82755449e-02,
       0.00000000e+00, 1.00000000e+00, 5.68680225e-01, 3.36398272e-03,
       0.00000000e+00, 3.21175678e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.75401845e-01, 0.00000000e+00, 0.00000000e+00,
       6.23039326e-01, 5.93133864e-01, 1.37295643e-02, 0.00000000e+00,
       6.89671686e-04, 0.00000000e+00, 0.00000000e+00, 1.75401845e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.35339851e-01,
       5.89743844e-01, 0.00000000e+00, 1.38373412e-02, 0.00000000e+00,
       0.00000000e+00, 5.82755449e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.55756934e-01, 3.21770853e-01,
       0.00000000e+00, 2.77966544e-02, 0.00000000e+00, 2.79481002e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.79255069e-01, 3.83246638e-01, 1.66299386e-03, 0.00000000e+00,
       1.32233763e-02, 0.00000000e+00, 5.02812637e-04, 1.00000000e+00,
      

In [201]:
fill_factor.argmax()

5

In [196]:
x[5]

0.025

In [197]:
y[5]

0.5

In [170]:
ff = np.zeros((11,121))
for i in range (11):
    ff[i] = U(i)
ff

Solving linear variational problem.
Q max:  0.21157248639103562
dt:     4.726512492516466
Solving linear variational problem.
Q max:  0.5955637281677462
dt:     1.6790814361319542
Solving linear variational problem.Q max: 
 0.5667501332954631
dt:     1.7644459899556324
Solving linear variational problem.
Q max:  0.5718720493014815
dt:     1.7486429022391625
Q max: Solving linear variational problem.
 0.5720470361922099
dt:     1.7481079993988404
Q max: Solving linear variational problem.
 0.5717326072558545
dt:     1.7490693854242474
Q max: Solving linear variational problem.
 0.5713937278341332
dt:     1.7501067150150527
Q max: Solving linear variational problem.
 0.5713979186831847
dt:     1.7500938790686364
Q max: Solving linear variational problem.
 0.5970876809588174
dt:     1.6747958999826904
Q max: Solving linear variational problem.
 0.5658107920989188
dt:     1.7673752674289276
Q max: Solving linear variational problem.
 0.8580212957073845
dt:     1.1654722382800102


array([[1.41175842e-01, 0.00000000e+00, 1.00000000e+00, ...,
        2.33266340e-02, 2.27942511e-02, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 5.03198535e-01, ...,
        0.00000000e+00, 0.00000000e+00, 5.94641027e-10],
       [2.55413979e-02, 0.00000000e+00, 1.00000000e+00, ...,
        1.61290062e-07, 0.00000000e+00, 2.33816073e-09],
       ...,
       [0.00000000e+00, 1.14379577e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.12212475e-05],
       [1.14324789e-06, 0.00000000e+00, 2.68166600e-05, ...,
        5.59922749e-03, 0.00000000e+00, 1.72638096e-04],
       [0.00000000e+00, 2.26419725e-06, 0.00000000e+00, ...,
        0.00000000e+00, 5.62055899e-04, 1.28279585e-04]])

In [188]:
heatmaps = [ff[i] for i in range(11)]
fig = go.Figure(data=go.Heatmap(x=x, y=y, z=heatmaps[0]),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(11)])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()